In [84]:
import math 
import string 
import re

import json 

import pandas as pd 
import sqlite3
import requests 
import bs4 

In [85]:
from query_config import pubmed_query_map

In [86]:
len(pubmed_query_map)

46

In [87]:

def connect_to_db(db_path):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    return conn, c 

In [228]:
parent_path = '/home/ramprasad.sa'
db_path = f'{parent_path}/human_annotations_factuality/XSUM_CNN/set1/annotated/news_summaries_set1_final.db'
conn, c = connect_to_db(db_path)
df_annotations = pd.read_sql('SELECT * from label', conn)
articles = list(set(df_annotations['article'].values))
articles = sorted(articles)


In [96]:
df_annotations[df_annotations['summary_uuid'] == '515677b3-58ae-4e33-b89f-a7e1b6659ce4_gpt3_gen']

      

,uuid,user_id,summary_uuid,summ_id,system_id,label_type,summary,nonfactual_sentences,article
68,69,ann_krcnbm,515677b3-58ae-4e33-b89f-a7e1b6659ce4_gpt3_gen,billsum_generic,gpt3,non_factual,The Social Security Act has been amended to re...,The Social Security Act has been amended to re...,SECTION 1. AMENDMENT.\r\n\r\n Section 403 o...
176,177,ann_hguilf,515677b3-58ae-4e33-b89f-a7e1b6659ce4_gpt3_gen,billsum_generic,gpt3,non_factual,The Social Security Act has been amended to re...,The amendment also requires that general welfa...,SECTION 1. AMENDMENT.\r\n\r\n Section 403 o...


In [280]:
from datasets import load_dataset
from nltk.tokenize import sent_tokenize

dataset = load_dataset("billsum", split="test", cache_dir = '/scratch/ramprasad.sa/huggingface_datasets')

def get_title(phrase):
    phrase = f'''{phrase}'''
    # print(phrase)
    all_titles = []
    for each in dataset:
        txt = each['text']
        txt =  txt.replace('\n', ' ').replace('\r', '')
        if phrase.strip() == txt.strip():
            all_titles.append(each['title'])
            # print('Title', each['title'])
            # print('Summary', each['summary'])
            # print(each['text'].lower())
            
            # print('**'*13)
    if len(all_titles) == 1:
        return all_titles[0]
    else:
        return None
        

domain_title_config = {}
for article in [articles[52]]:
    df_art = df_annotations[df_annotations['article'] == article]
    summary_uuids = list(set(df_art['summary_uuid'].values))
    article = article.replace('\n', ' ').replace('\r', '')
    # title = get_title(article)
    # if title:
    #     domain_title_config[title] = summary_uuid
    # else:
    #     print(summary_uuids)
    print(list(set(df_art['summary_uuid'].values)))
    print(article)

Using custom data configuration default
Reusing dataset billsum (/scratch/ramprasad.sa/huggingface_datasets/billsum/default/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959)


IndexError: list index out of range

In [ ]:
dataset[0]


In [52]:
news_quer_map = {
    
}

In [84]:
dataset

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 3269
})

In [93]:
# list(zip(df_annotations['summary_uuid'].values, df_annotations['article'].values))

In [94]:
# query_map = {
#     '6662e68fa9d78198ba62bc347bfedd63dd9c6510_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=BMW+road+rage+AND+east+London+AND+shirtless+man',
    
#     '6662e68fa9d78198ba62bc347bfedd63dd9c6510_flant5_gen': 'https://c4-search.apps.allenai.org/?q=BMW+road+rage+AND+east+London+AND+shirtless+man',
    
#     'd987468e948d4b351423c1d30a0db202545e879c_gpt3_gen':'https://c4-search.apps.allenai.org/?q=Anti-Hillary+Clinton+street+art++AND+Brooklyn+AND+presidential+candidacy+announcement',
    
#     'd987468e948d4b351423c1d30a0db202545e879c_flant5_gen':'https://c4-search.apps.allenai.org/?q=Anti-Hillary+Clinton+street+art++AND+Brooklyn+AND+presidential+candidacy+announcement',
    
#     '30840160_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Belgian+anti-terror+raid+AND+Verviers+AND+four+Kalashnikovs',
   
#     '30840160_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Belgian+anti-terror+raid+AND+Verviers+AND+four+Kalashnikovs',
    
#     '34699414_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Steve+Silberman+AND+autism+book+AND+Samuel+Johnson+prize',
    
#     '34699414_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Steve+Silberman+AND+autism+book+AND+Samuel+Johnson+prize',
    
#     '34276220_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Gareth+Edwards+AND+knighted+AND+Windsor+Castle',
    
#     '34276220_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Gareth+Edwards+AND+knighted+AND+Windsor+Castle',
    
#     '39188164_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Casey+Anthony+AND+breaks+silence+AND+Orlando+mother',
    
#     '39188164_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Casey+Anthony+AND+breaks+silence+AND+Orlando+mother',
    
#     '29911712_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Bradford+AND+Hanson+Academy+AND+uniform',
    
#     '29911712_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Bradford+AND+Hanson+Academy+AND+uniform',
    
#     '39033879_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Half-back+AND+Marc+Sneyd+AND+contract',
    
#     '39033879_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Half-back+AND+Marc+Sneyd+AND+contract',
    
#     '40825915_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Bristol+Rovers+AND+Ryan+Sweeney+AND+Dominic+Telford',
    
#     '40825915_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Bristol+Rovers+AND+Ryan+Sweeney+AND+Dominic+Telford',
    
#     '30002988_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Adoption+AND+Leadership+AND+Board+AND+James+AND+Munby+AND+2014',
    
#     '30002988_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Adoption+AND+Leadership+AND+Board+AND+James+AND+Munby+AND+2014',
    
#     '8bb0d16f2a8ef6f9592d4e539690e666f798e4ba_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=3D+AND+map+AND+German+AND+Aerospace+AND+Center+AND+Mars',
    
#     '8bb0d16f2a8ef6f9592d4e539690e666f798e4ba_flant5_gen': 'https://c4-search.apps.allenai.org/?q=3D+AND+map+AND+German+AND+Aerospace+AND+Center+AND+Mars',
    
#     '32691589_flant5_gen' : 'https://c4-search.apps.allenai.org/?q=Transatlantic+AND+Trade+AND+Investment+AND+Partnership+AND+Obama+AND+new+AND+jobs+AND+atlantic',
    
#     '32691589_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Transatlantic+AND+Trade+AND+Investment+AND+Partnership+AND+Obama+AND+new+AND+jobs+AND+atlantic',
    
#     '2e1b50654f3624bc4e671ffb652694c82a2a4115_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=skin+AND+difficult+AND+tear+AND+collagen+AND+Berkeley',
    
#     '2e1b50654f3624bc4e671ffb652694c82a2a4115_flant5_gen': 'https://c4-search.apps.allenai.org/?q=skin+AND+difficult+AND+tear+AND+collagen+AND+Berkeley',
    
    
# }

In [ ]:
# pubmed_query_map = {
# #     'ab30d3f8-09fe-4d9a-aaad-b7227cfeab74_flant5_gen': 'https://c4-search.apps.allenai.org/?q=java+AND+linkage+AND+disequilibrium+AND+plotter',
    
# #     'ab30d3f8-09fe-4d9a-aaad-b7227cfeab74_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=java+AND+linkage+AND+disequilibrium+AND+plotter',
    
# #     'e6a6a5ec-b9f0-4e48-954f-1207f7750d64_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Induction+AND+temperate+AND+cyanophage+AND+metal',
    
# #     'e6a6a5ec-b9f0-4e48-954f-1207f7750d64_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Induction+AND+temperate+AND+cyanophage+AND+metal',
    
# #     '38689e3f-0932-4a5e-9820-dfc4e8a39bfb_flant5_gen': 'https://c4-search.apps.allenai.org/?q=Enrichment+AND+microbial+AND+anaerobic+AND+oxidation+AND+methane+AND+continuous+AND+high+AND+pressure+AND+bioreactor',
    
# #     '38689e3f-0932-4a5e-9820-dfc4e8a39bfb_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=Enrichment+AND+microbial+AND+anaerobic+AND+oxidation+AND+methane+AND+continuous+AND+high+AND+pressure+AND+bioreactor',
    
# #     '22e2e9ef-c515-4f2c-89a2-8e5d8792a530_flant5_gen': 'https://c4-search.apps.allenai.org/?q=+PCR+AND+based+AND+method+AND+high+AND+throughput+AND+prokaryotic+AND+expression+AND+antimicrobial+AND+peptide+AND+genes',
    
# #     '22e2e9ef-c515-4f2c-89a2-8e5d8792a530_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=+PCR+AND+based+AND+method+AND+high+AND+throughput+AND+prokaryotic+AND+expression+AND+antimicrobial+AND+peptide+AND+genes',
    
# #     '05bbed11-24ff-4f82-b465-e701377361a9_flant5_gen': 'https://c4-search.apps.allenai.org/?q=SIGMA+AND+System+AND+Integrative+AND+Genomic+AND+Microarray+AND+Analysis+AND+Cancer+AND+Genomes',
    
# #     '05bbed11-24ff-4f82-b465-e701377361a9_gpt3_gen': 'https://c4-search.apps.allenai.org/?q=SIGMA+AND+System+AND+Integrative+AND+Genomic+AND+Microarray+AND+Analysis+AND+Cancer+AND+Genomes'

# # }